In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
phone=pd.read_csv(r'C:\Users\adaks\AppData\Local\Programs\Python\Python312\Scripts\Phone Price Prediction\Phone_cleaned_dataset.csv')

In [3]:
phone.head()

,Unnamed: 0,Brand me,Ratings,RAM,Mobile_Size,Primary_Cam,Selfi_Cam,Battery_Power,Price
0,0,LG,4.3,4.0,6.00,48,13,4000,24999.0
1,1,IKall,3.4,6.0,4.50,48,12,4000,15999.0
2,2,Nokia,4.3,4.0,4.50,64,16,4000,15000.0
3,3,Samsung,4.4,6.0,6.40,48,15,3800,18999.0
4,4,POCO,4.5,6.0,6.18,35,15,3800,18999.0


In [4]:
phone.shape

(521, 9)

In [5]:
phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     521 non-null    int64  
 1   Brand me       521 non-null    object 
 2   Ratings        521 non-null    float64
 3   RAM            521 non-null    float64
 4   Mobile_Size    521 non-null    float64
 5   Primary_Cam    521 non-null    int64  
 6   Selfi_Cam      521 non-null    int64  
 7   Battery_Power  521 non-null    int64  
 8   Price          521 non-null    float64
dtypes: float64(4), int64(4), object(1)
memory usage: 36.8+ KB


In [6]:
backup=phone.copy()

Converting brand name into numerical dtype using one hot encoding

In [7]:
X = phone[['Brand me', 'RAM','Mobile_Size', 'Primary_Cam', 'Selfi_Cam', 'Battery_Power']]
y = phone['Price']

In [8]:
X

,Brand me,RAM,Mobile_Size,Primary_Cam,Selfi_Cam,Battery_Power
0,LG,4.0,6.00,48,13,4000
1,IKall,6.0,4.50,48,12,4000
2,Nokia,4.0,4.50,64,16,4000
3,Samsung,6.0,6.40,48,15,3800
4,POCO,6.0,6.18,35,15,3800
...,...,...,...,...,...,...
516,Samsung,4.0,6.20,35,8,3500
517,Black,6.0,6.39,48,12,3800
518,Karbonn,6.0,4.54,48,12,2800
519,Alcatel,3.0,6.20,48,1,3800


In [9]:
y

0      24999.0
1      15999.0
2      15000.0
3      18999.0
4      18999.0
        ...   
516    53990.0
517    31999.0
518     1299.0
519     9790.0
520      799.0
Name: Price, Length: 521, dtype: float64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2)

Converting Brand column into numerical data using one hot encoding

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [12]:
ohe=OneHotEncoder()
ohe.fit(X[['Brand me']])

OneHotEncoder()

In [14]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Brand me']),remainder='passthrough',force_int_remainder_cols=False)

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [16]:
model=DecisionTreeRegressor()
pipe=make_pipeline(column_trans,model)
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Alcatel', 'Apple', 'Blacear', 'Blacerry', 'Black', 'BlackZone',
       'Doogee', 'Easyfone', 'Forme', 'Good', 'Google', 'Grabo',
       'GreenBerry', 'Hicell', 'Honor', 'Huawei', 'I', 'IKall', 'InFocus',
       'Inf...ovu', 'Itel', 'Jivi', 'Jmax', 'Karbonn', 'Kechaoda',
       'LG', 'Lava', 'Lenovo', 'MTR', 'Mafe', 'Megus', 'Meizu', 'Mi',
       'Micax', 'Moto', 'Motorola', 'Muphone', 'Nexus', 'Nokia', 'OPPO',
       'OnePlus', 'POCO', 'Peace', 'Q-Tel', 'Realme', 'Redmi', 'Salora',
       'Samsung', 'Ssky', 'Tecno', 'Tork', 'Vivo', 'iQOO'], dtype=object)]),
                                                  ['Brand me'])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

Our model has been trained!

In [17]:
y_pred=pipe.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error(MSE):{mse}")

Mean Squared Error(MSE):228218809.8285265


In [19]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,y_pred)
print(f"R-square score(R²):{r2}")

R-square score(R²):0.6166651645411345


Now we will find the random state where the r2 score comes out to be the maximum

In [20]:
scores=[]
for i in range (100):
    X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=i)
    column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Brand me']),remainder='passthrough',force_int_remainder_cols=False)
    model=DecisionTreeRegressor()
    pipe=make_pipeline(column_trans,model)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [21]:
np.argmax(scores)

96

In [23]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=np.argmax(scores))
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Brand me']),remainder='passthrough',force_int_remainder_cols=False)
model=DecisionTreeRegressor()
pipe=make_pipeline(column_trans,model)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.9973119245428052

In [25]:
import pickle

In [26]:
pickle.dump(pipe,open('DecisionTreeModelPhone','wb'))

In [27]:
pipe.predict(pd.DataFrame([['Samsung',6.0,6.4,48,16,3600]],columns=['Brand me','RAM','Mobile_Size','Primary_Cam','Selfi_Cam','Battery_Power']))

array([18999.])

Our Model works perfectly fine!